In [1]:
import numpy as np
import pandas as pd
from grad_boost import GradientBooster
from sklearn.preprocessing import StandardScaler
from dataset import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
X,y = load_dataset()
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42) 

In [3]:
#scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.array(y_train).reshape(X_train.shape[0],1)
y_test = np.array(y_test).reshape(X_test.shape[0],1)

In [4]:
g = GradientBooster()
losses = g.train(X_train,y_train)

KeyboardInterrupt: 

In [12]:
pred = g.predict(X_test)
mean_squared_error(pred,y_test)

1460.3586305872002